In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import glob
from human_body_prior.tools.omni_tools import copy2cpu as c2c

In [2]:
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from human_body_prior.body_model.body_model import BodyModel

bm_path = '/home/michalislazarou/PhD/amass/smplh/male/model.npz'
dmpl_path = '/home/michalislazarou/PhD/amass/dmpl/male/model.npz'
npz_bdata_path ='/home/michalislazarou/PhD/amass/datasets/EKUT/234/'
num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

bm = BodyModel(bm_path=bm_path, num_betas=num_betas, num_dmpls=num_dmpls, path_dmpl=dmpl_path).to(comp_device)
faces = c2c(bm.f)

In [7]:
# import meta-dataset
data_list = []
for data_file in glob.glob(os.path.join(npz_bdata_path, '*.npz')):
    bdata = dict(np.load(data_file))
    bdata['poses'] = bdata['poses'][:200,:]
    bdata['trans'] = bdata['trans'][:200,:]
    bdata['dmpls'] = bdata['dmpls'][:200,:]
    data_list.append(bdata)
train_list, test_list = data_list[:65], data_list[65:] 
print(len(data_list), len(train_list), len(test_list))

79 65 14
